In [1]:
import sys
sys.path.append("/kaggle/input/marigold_model/transformers/default/2/marigold_source")
from Marigold.marigold.marigold_depth_pipeline import MarigoldDepthPipeline

2025-08-05 15:01:05.792699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754406065.985702      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754406066.043985      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os, gc, cv2, torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import shutil
from PIL import Image
from tqdm import tqdm

# moxingjiazai
model_path = "/kaggle/input/marigold_model/transformers/default/2/marigold_model"
pipe = MarigoldDepthPipeline.from_pretrained(model_path).to("cuda")

# ========== 路径配置 ==========
base_dir = "/kaggle/input/kitti-dataset/depth_selection/val_selection_cropped"
img_dir = os.path.join(base_dir, "image")
gt_dir = os.path.join(base_dir, "groundtruth_depth")

save_dir = "/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)
vis_dir = os.path.join(save_dir, "visualizations")
os.makedirs(vis_dir, exist_ok=True)
pres_dir = os.path.join(save_dir, "pres")
os.makedirs(pres_dir, exist_ok=True)

# ========== 可视化函数 ==========
def save_comparison_figure(rgb, gt_depth, pred_depth_aligned, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt_depth > 0) & np.isfinite(gt_depth) & np.isfinite(pred_depth_aligned)
    if np.sum(valid) == 0:
        vmin, vmax = 0, 1  # fallback, no valid values
    else:
        all_depths = np.concatenate([gt_depth[valid], pred_depth_aligned[valid]])
        vmin = np.percentile(all_depths, 2)
        vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1)
    plt.imshow(rgb)
    plt.title("RGB")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt_depth, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred_depth_aligned, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# ========== 评估函数 ==========
def compute_abs_rel(pred, gt, mask=None):
    # 支持 mask=None
    if mask is None:
        valid = (gt > 0)
    else:
        valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    return np.mean(np.abs(pred - gt) / gt)

def compute_delta1(pred, gt, mask):
    valid_mask = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid_mask], gt[valid_mask]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    denominator = np.sum(pred ** 2)
    return np.sum(gt * pred) / denominator if denominator > 1e-8 else 1.0

# ========== 推理流程 ==========
results = []
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".png")])

for img_file in tqdm(img_files, desc="Evaluating"):
    try:
        rgb_path = os.path.join(img_dir, img_file)
        gt_file = re.sub("image", "groundtruth_depth", img_file, count=1)
        gt_path = os.path.join(gt_dir, gt_file)

        # 读取图像
        rgb = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        input_image = Image.fromarray(rgb)

        # 读取 GT
        gt_png = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        gt_depth = gt_png / 256.0
        mask = gt_depth > 0

        with torch.no_grad():
            result = pipe(
                input_image,
                denoising_steps=4,
                ensemble_size=1,
                processing_res=768,
                match_input_res=True,
                batch_size=0,
                show_progress_bar=False,
            )
        pred_depth = result.depth_np

        # resize 到 GT 尺寸
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

        # scale 匹配
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale

        # 评估指标
        absrel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1 = compute_delta1(pred_aligned, gt_depth, mask)
        results.append([img_file, absrel, delta1])
        print(f"[{img_file}] AbsRel: {absrel:.4f}, δ1: {delta1 * 100:.2f}%")

        # 可视化保存
        vis_path = os.path.join(vis_dir, img_file.replace(".png", "_vis.png"))
        save_comparison_figure(rgb, gt_depth, pred_aligned, mask, vis_path)

        # 保存深度图
        np.save(os.path.join(pres_dir, img_file.replace(".png", "_pred.npy")), pred_aligned)

        del rgb, gt_depth, pred_depth, pred_aligned
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"跳过 {img_file}，错误：{e}")
        continue

# ========== 保存评估结果 ==========
df = pd.DataFrame(results, columns=["image", "AbsRel", "Delta1"])
df.to_csv(os.path.join(save_dir, "eval_results.csv"), index=False)
print(f"\n所有评估结果保存至：{os.path.join(save_dir, 'eval_results.csv')}")

# 打包
zip_path = shutil.make_archive(os.path.join(save_dir, "visualizations_all"), 'zip', root_dir=vis_dir)
print(f"可视化图像已打包: {zip_path}")
pres_zip_path = shutil.make_archive(os.path.join(save_dir, "pres"), 'zip', root_dir=pres_dir)
print(f"pres 已打包至: {pres_zip_path}")


The config attributes {'prediction_type': 'depth'} were passed to MarigoldDepthPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'prediction_type': 'depth'} are not expected by MarigoldDepthPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

[2011_09_26_drive_0002_sync_image_0000000005_image_02.png] AbsRel: 0.3189, δ1: 42.53%


Evaluating:   0%|          | 1/1000 [00:02<44:57,  2.70s/it]

...

[2011_10_03_drive_0047_sync_image_0000000785_image_02.png] AbsRel: 0.3112, δ1: 43.17%


Evaluating: 100%|█████████▉| 999/1000 [59:54<00:05,  5.55s/it]

[2011_10_03_drive_0047_sync_image_0000000791_image_03.png] AbsRel: 0.3806, δ1: 29.28%


Evaluating: 100%|██████████| 1000/1000 [1:00:00<00:00,  3.60s/it]



所有评估结果保存至：/kaggle/working/eval_results.csv
可视化图像已打包: /kaggle/working/visualizations_all.zip
pres 已打包至: /kaggle/working/pres.zip


In [7]:
import pandas as pd

# 读取CSV
csv_path = '/kaggle/working/eval_results.csv'  # 路径按实际调整
df = pd.read_csv(csv_path)

# 计算均值
mean_absrel = df['AbsRel'].mean()
mean_delta1 = df['Delta1'].mean()

print(f"AbsRel 均值: {mean_absrel:.4f}")
print(f"Delta1 均值: {mean_delta1:.4f}")

result = pd.DataFrame([{'all_mean': 'KITTI','AbsRel': mean_absrel, 'Delta1': mean_delta1}])
result.to_csv('/kaggle/working/eval_results_mean.csv', index=False)
print("均值已保存到 eval_results_mean.csv")


AbsRel 均值: 0.3115
Delta1 均值: 0.4230
均值已保存到 eval_results_mean.csv
